In [2]:
import json
import os
import time
from pandas import json_normalize
import psycopg2
from psycopg2 import sql
import numpy as np

import pandas as pd
import requests
from dotenv import load_dotenv

load_dotenv()

access_token = os.getenv("ACCESS_TOKEN")
HOST = os.getenv("HOST")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")


# Informações de conexão com o banco de dados PostgreSQL
db_config = {
    "host": HOST,
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
}

# Registra o tempo antes da execução
start_prog = time.time()

In [3]:
print(access_token)

APP_USR-6628000663056293-112306-60f950a7239840738ac26b14f0e36ca2-233632476


### Verificando dados da conta

In [4]:
url = "https://api.mercadolibre.com/users/me"

payload = {}
headers = {
  'Authorization': f'Bearer {access_token}'
}

response = requests.request("GET", url, headers=headers, data=payload)
response = response.text

print(response)

# Encontre posição do início e fim do campo "id"
start_pos = response.find('"id"')  # Encontra onde começa "id"
end_pos = response.find(',', start_pos)  # Encontra onde termina "id"

# Extraia o valor de "id"
seller_id = response[start_pos:end_pos].split(":")[1]
seller_id = seller_id.strip().strip('"')

print(f'O valor de "seller_id" é: {seller_id}')

{"id":233632476,"nickname":"MUSICALCENTERPG","registration_date":"2016-11-05T17:35:04.000-04:00","first_name":"MUSICAL CENTER PONTA","last_name":"GROSSA EIRELI -","gender":"","country_id":"BR","email":"musicalc168@gmail.com","identification":{"number":"10885874000127","type":"CNPJ"},"address":{"address":"Estrada da Gruta 775","city":"Louveira","state":"BR-SP","zip_code":"13290431"},"phone":{"area_code":"19","extension":"","number":"38787702","verified":false},"alternative_phone":{"area_code":"","extension":"","number":""},"user_type":"brand","tags":["brand","large_seller","eshop","mshops","messages_as_seller"],"logo":null,"points":4528,"site_id":"MLB","permalink":"http://perfil.mercadolivre.com.br/MUSICALCENTERPG","seller_experience":"ADVANCED","bill_data":{"accept_credit_note":"Y"},"seller_reputation":{"level_id":"5_green","power_seller_status":"platinum","transactions":{"canceled":1537,"completed":33428,"period":"historic","ratings":{"negative":0.05,"neutral":0.03,"positive":0.92},"t

### Carregando Inventory Ids de Produtos a serem pesquisados

Esses dados podem ser encontrados na tabela *tiny_fulfillment* do banco de dados

In [5]:
conn = psycopg2.connect(**db_config)

sql_query = "SELECT * FROM tiny_fulfillment"

df_codes = pd.read_sql(sql_query, conn)

df_codes.head(1)


/tmp/ipykernel_138991/3276080531.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_codes = pd.read_sql(sql_query, conn)


,mcenter_id,ml_inventory_id,ml_code,ml_sku,var_code,tiny_id,tiny_sku,ad_title,created_at,updated_at
0,1,DSGP06967,1992567302,FULLNI5BM,1 Par 5B,735947207,7897937421021,1 Par Baquetas De Bateria Liverpool Nirvana 5a...,2023-11-21 20:19:26.940770,None


In [6]:
# Verifica se há duplicatas
duplicates = df_codes.duplicated()
n_dup = duplicates.sum()

print(f'Número de duplicatas: {n_dup}')

# Encontre as duplicatas no DataFrame
duplicates = df_codes[df_codes.duplicated(keep=False)]

# Exiba as próprias duplicatas
duplicates

Número de duplicatas: 0


,mcenter_id,ml_inventory_id,ml_code,ml_sku,var_code,tiny_id,tiny_sku,ad_title,created_at,updated_at


In [7]:
# Remove as duplicatas e atualiza o DataFrame
df_codes = df_codes.drop_duplicates()

df_codes.shape

(872, 10)

In [8]:
df_codes.head(1)

,mcenter_id,ml_inventory_id,ml_code,ml_sku,var_code,tiny_id,tiny_sku,ad_title,created_at,updated_at
0,1,DSGP06967,1992567302,FULLNI5BM,1 Par 5B,735947207,7897937421021,1 Par Baquetas De Bateria Liverpool Nirvana 5a...,2023-11-21 20:19:26.940770,None


### Pegando Estoque de FulFillment

In [9]:
# removendo valores nulos
df_codes = df_codes.dropna(subset=['ml_inventory_id'])
df_codes['ml_inventory_id'].value_counts()

ml_inventory_id
NaN          82
KMCE82136     5
DMTU81740     5
FSNB76403     5
ITDN80392     5
             ..
ACBY18022     1
BHEZ89925     1
SJIT31691     1
WRRA65769     1
RKCI54992     1
Name: count, Length: 636, dtype: int64

In [10]:
codes = df_codes['ml_inventory_id'].unique()
print(len(codes))

636


In [11]:
x = 'KMCE82136'
y = df_codes[df_codes['ml_inventory_id']== x]
y

,mcenter_id,ml_inventory_id,ml_code,ml_sku,var_code,tiny_id,tiny_sku,ad_title,created_at,updated_at
450,450,KMCE82136,1943185413,FULLKIT3,NaN,746248995,RH-01GD,Kit Straplock+afinador+capotraste+6 Palhetas+ ...,2023-11-21 20:19:26.940770,None
451,451,KMCE82136,1943185413,FULLKIT3,NaN,565665071,Palheta,Kit Straplock+afinador+capotraste+6 Palhetas+ ...,2023-11-21 20:19:26.940770,None
452,452,KMCE82136,1943185413,FULLKIT3,NaN,565663877,PPalheta,Kit Straplock+afinador+capotraste+6 Palhetas+ ...,2023-11-21 20:19:26.940770,None
453,453,KMCE82136,1943185413,FULLKIT3,NaN,509978212,Bracadeirakits,Kit Straplock+afinador+capotraste+6 Palhetas+ ...,2023-11-21 20:19:26.940770,None
454,454,KMCE82136,1943185413,FULLKIT3,NaN,597415799,AFINADORKIT,Kit Straplock+afinador+capotraste+6 Palhetas+ ...,2023-11-21 20:19:26.940770,None


In [12]:
len(codes)

636

In [13]:
import numpy as np

codes = np.delete(codes, np.where(codes == 'NaN'))
codes

array(['DSGP06967', 'DSGP06979', 'JJSZ06277', 'DGCL82617', 'GCYF86172',
       'SIZW84848', 'SOQF03130', 'ZRDQ91074', 'TAPQ85749', 'FRBW82060',
       'IYUQ81169', 'LUHT78439', 'XUMC81097', 'XYRN86487', 'EDHV29406',
       'EJRL85435', 'EMJF86603', 'GUSW29256', 'OQVC17160', 'LCUY78585',
       'ITMG31457', 'UJHN75001', 'MENA63204', 'DCPP47589', 'FRPG47185',
       'SKXW47953', 'VGBX48763', 'YQFX46259', 'YQWM74245', 'TCHX70973',
       'KBOK20943', 'ELKX63703', 'ZXFT21237', 'WGMG44862', 'HGUJ21007',
       'LNLA50194', 'GMII57127', 'MYSJ73106', 'PIJL35032', 'WXUT71108',
       'HXTD35130', 'EDGX70669', 'NDSR34802', 'CHXM44815', 'VBZZ52467',
       'NOMF53205', 'MNSV53924', 'JGED52238', 'CIKE94068', 'AZVM53003',
       'WCNJ53438', 'ROAL74330', 'UCYX71938', 'OXVU12317', 'ELKB12099',
       'YWET11585', 'QHVP51310', 'ZOQD52873', 'YWET52744', 'LGAM52077',
       'WTQW54410', 'UKNM52020', 'UWGO53377', 'BESH52678', 'WFEX53455',
       'FFIA52697', 'HFNJ52229', 'UIUW52962', 'JLFV52818', 'YWET

In [14]:
len(codes)

635

In [15]:

unique_values = set(codes)
num_unique_values = len(unique_values)

print(f"Quantidade de valores únicos: {num_unique_values}")

Quantidade de valores únicos: 635


In [16]:
counter = 0
json_list = []

for item in codes:
    url = f"https://api.mercadolibre.com/inventories/{item}/stock/fulfillment"

    payload = {}
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    print(f'Buscando dados {counter}/{len(codes)}: {item}')

    try:
        response = requests.get(url, headers=headers, data=payload)
        response.raise_for_status()  # Lança uma exceção HTTPError para códigos de status de erro
        response_data = response.json()
        json_list.append(response_data)
        counter += 1

        if counter % 50 == 0:
            print(f"Fazendo uma pausa de 1 minuto...")
            time.sleep(60)

    except requests.exceptions.RequestException as e:
        print(f"Erro ao fazer a requisição para {url}: {e}")

    except Exception as e:
        print(f"Erro não esperado: {e}")


Buscando dados 0/635: DSGP06967
Buscando dados 1/635: DSGP06979
Buscando dados 2/635: JJSZ06277
Buscando dados 3/635: DGCL82617
Buscando dados 4/635: GCYF86172
Buscando dados 5/635: SIZW84848
Buscando dados 6/635: SOQF03130
Buscando dados 7/635: ZRDQ91074
Buscando dados 8/635: TAPQ85749
Buscando dados 9/635: FRBW82060
Buscando dados 10/635: IYUQ81169
Buscando dados 11/635: LUHT78439
Buscando dados 12/635: XUMC81097
Buscando dados 13/635: XYRN86487
Buscando dados 14/635: EDHV29406
Buscando dados 15/635: EJRL85435
Buscando dados 16/635: EMJF86603
Buscando dados 17/635: GUSW29256
Buscando dados 18/635: OQVC17160
Buscando dados 19/635: LCUY78585
Buscando dados 20/635: ITMG31457
Buscando dados 21/635: UJHN75001
Buscando dados 22/635: MENA63204
Buscando dados 23/635: DCPP47589
Buscando dados 24/635: FRPG47185
Buscando dados 25/635: SKXW47953
Buscando dados 26/635: VGBX48763
Buscando dados 27/635: YQFX46259
Buscando dados 28/635: YQWM74245
Buscando dados 29/635: TCHX70973
Buscando dados 30/63

In [17]:
len(json_list)

635

In [18]:
print(json_list)

[{'inventory_id': 'DSGP06967', 'total': 6, 'available_quantity': 6, 'not_available_quantity': 0, 'not_available_detail': [], 'external_references': [{'type': 'item', 'id': 'MLB1992567302', 'variation_id': 173585284762}]}, {'inventory_id': 'DSGP06979', 'total': 8, 'available_quantity': 7, 'not_available_quantity': 1, 'not_available_detail': [{'status': 'noFiscalCoverage', 'quantity': 1}], 'external_references': [{'type': 'item', 'id': 'MLB1992567302', 'variation_id': 173585284763}]}, {'inventory_id': 'JJSZ06277', 'total': 48, 'available_quantity': 48, 'not_available_quantity': 0, 'not_available_detail': [], 'external_references': [{'type': 'item', 'id': 'MLB1992567302', 'variation_id': 173585284761}]}, {'inventory_id': 'DGCL82617', 'total': 13, 'available_quantity': 13, 'not_available_quantity': 0, 'not_available_detail': [], 'external_references': [{'type': 'item', 'id': 'MLB2606294854', 'variation_id': 174383750598}]}, {'inventory_id': 'GCYF86172', 'total': 12, 'available_quantity': 1

In [19]:
df = pd.DataFrame(json_list)
df

,inventory_id,total,available_quantity,not_available_quantity,not_available_detail,external_references
0,DSGP06967,6,6,0,[],"[{'type': 'item', 'id': 'MLB1992567302', 'vari..."
1,DSGP06979,8,7,1,"[{'status': 'noFiscalCoverage', 'quantity': 1}]","[{'type': 'item', 'id': 'MLB1992567302', 'vari..."
2,JJSZ06277,48,48,0,[],"[{'type': 'item', 'id': 'MLB1992567302', 'vari..."
3,DGCL82617,13,13,0,[],"[{'type': 'item', 'id': 'MLB2606294854', 'vari..."
4,GCYF86172,12,12,0,[],"[{'type': 'item', 'id': 'MLB2606294854', 'vari..."
...,...,...,...,...,...,...
630,ERIM51807,1,0,1,"[{'status': 'lost', 'quantity': 1}]","[{'type': 'item', 'id': 'MLB949788598'}]"
631,TDFV51283,0,0,0,[],"[{'type': 'item', 'id': 'MLB950297972'}]"
632,OYUK51882,16,16,0,[],"[{'type': 'item', 'id': 'MLB1363602423'}]"
633,BLHH53768,4,4,0,[],"[{'type': 'item', 'id': 'MLB2004836143'}]"


In [20]:
df_full = df[['inventory_id', 'available_quantity']]
df_full

,inventory_id,available_quantity
0,DSGP06967,6
1,DSGP06979,7
2,JJSZ06277,48
3,DGCL82617,13
4,GCYF86172,12
...,...,...
630,ERIM51807,0
631,TDFV51283,0
632,OYUK51882,16
633,BLHH53768,4


In [21]:
### AMBIENTE INICIAL ###
### Repetindo os dados com dias diferentes
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()
import datetime  

custom_created_at = datetime.datetime(2023, 11, 20, 1, 0, 0)  

for _ in range(30): # Repetir 30 vezes para adicionar dados com datas diferentes
    for index, row in df_full.iterrows(): # preenchendo db com dados de estoque
            insert_query = sql.SQL("""
                INSERT INTO fulfillment_stock_hist (ml_inventory_id, available_quantity, created_at)
                VALUES (%s, %s, %s)
            """)
            cursor.execute(insert_query, (row['inventory_id'], row['available_quantity'], custom_created_at))
    custom_created_at += datetime.timedelta(days=-1)


conn.commit()

cursor.close()
conn.close()

print('Dados inseridos com sucesso!')

Dados inseridos com sucesso!


In [ ]:
### AMBIENTE DE PRODUÇÃO ###
### Adicionando dados de estoque

conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

for index, row in df_full.iterrows():
    insert_query = sql.SQL("""
        INSERT INTO fulfillment_stock_hist (ml_inventory_id, available_quantity)
        VALUES (%s, %s)
    """)
    cursor.execute(insert_query, (row['inventory_id'], row['available_quantity']))

conn.commit()

cursor.close()
conn.close()

print('Dados inseridos com sucesso!')

## É preciso o restante?

In [ ]:
df_er = json_normalize(json_list, record_path='external_references', meta=['inventory_id', 'total', 'available_quantity', 'not_available_quantity', 'not_available_detail'])
df_nad = json_normalize(json_list, record_path='not_available_detail', meta=['inventory_id', 'total', 'available_quantity', 'not_available_quantity', 'external_references'])


print('df',df.shape)
print('df_er',df_er.shape)
print('df_nad',df_nad.shape)

df_er.head(3)

In [ ]:
df_nad.head(3)

In [ ]:
print(df_nad.columns)
print(df_er.columns)

In [ ]:
df_nad_ = df_nad.drop(columns='external_references')
df_er_ = df_er.drop(columns='not_available_detail')

In [ ]:
common_cols = ['inventory_id', 'total', 'available_quantity', 'not_available_quantity']
# df_fulfillment = df_er_.merge(df_nad_, on=common_cols, how='left', suffixes=('_er','_nad'))
df_fulfillment = df_er_.merge(df_nad_, on=common_cols, how='left')

df_fulfillment.shape

In [ ]:
df_fulfillment.head(1)

In [ ]:
map_cols = {'inventory_id': 'ml_inventory_id', 'id': 'ml_item_id', 'status': 'nad_status','quantity':'nad_quantity'}
df_fulfillment = df_fulfillment.rename(columns=map_cols)
order_col = ['ml_inventory_id','ml_item_id','variation_id','nad_status','nad_quantity','total','available_quantity','not_available_quantity','type']
df_fulfillment = df_fulfillment[order_col]

df_fulfillment.head(1)

In [ ]:
df_fulfillment.shape

In [ ]:
df_fulfillment.info()

In [ ]:
df_fulfillment['variation_id'] = df_fulfillment['variation_id'].astype(str)
df_fulfillment['nad_quantity'] = df_fulfillment['nad_quantity'].fillna(0).astype(int)

In [ ]:
df_fulfillment.info()

In [ ]:
import psycopg2
from psycopg2 import sql

# Suponha que você já tenha seu DataFrame df_fulfillment

# Conecte-se ao banco de dados
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

# Itere pelas linhas do DataFrame e insira os dados na tabela
for index, row in df_fulfillment.iterrows():
    insert_query = sql.SQL("""
        INSERT INTO ml_fulfillment3 (ml_inventory_id, ml_item_id, variation_id, nad_status, nad_quantity, total, available_quantity, not_available_quantity, type)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """)
    cursor.execute(insert_query, (row['ml_inventory_id'], row['ml_item_id'], row['variation_id'], row['nad_status'], row['nad_quantity'], row['total'], row['available_quantity'], row['not_available_quantity'], row['type']))

# Confirme as alterações
conn.commit()

# Feche o cursor e a conexão
cursor.close()
conn.close()

print('Dados inseridos com sucesso!')


## Formatando

In [ ]:

# pegando codigos
conn = psycopg2.connect(**db_config)

sql_query = "SELECT * FROM tiny_ml_codes"

df_codes = pd.read_sql(sql_query, conn)

codes = df_codes['ml_inventory_id'].unique()

counter = 0
json_list = []

# diminuir codes

for item in codes:
    url = f"https://api.mercadolibre.com/inventories/{item}/stock/fulfillment"

    payload = {}
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    print(f'Buscando dados de: {item}')

    response = requests.get(url, headers=headers, data=payload)
    response_data = response.json()

    json_list.append(response_data)
    
    counter += 1
    
    if counter % 50 == 0:
        print(f"Fazendo uma pausa de 1 minuto...")
        time.sleep(60)
